In [1]:
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option('display.float_format',lambda x: '{:.2f}'.format(x))

df = pd.read_csv('./data/data.csv')
df

,Borrower Name,Default Probability,Default Truth
0,Priya Rao,0.32,0
1,Raj Patel,0.67,1
2,Meera Gupta,0.56,1
3,Linda Johnson,0.18,0
4,Aditi Sharma,0.75,1
5,John Smith,0.49,1
6,Arjun Reddy,0.12,0
7,Emily Chen,0.44,0
8,Laura Kim,0.06,0
9,Vivek Singh,0.39,0


In [2]:
df_sorted = df.sort_values(by='Default Probability',ascending=False)
df_sorted

,Borrower Name,Default Probability,Default Truth
4,Aditi Sharma,0.75,1
1,Raj Patel,0.67,1
12,Anil Kumar,0.62,0
2,Meera Gupta,0.56,1
13,Sarah Lee,0.52,1
5,John Smith,0.49,1
7,Emily Chen,0.44,0
9,Vivek Singh,0.39,0
0,Priya Rao,0.32,0
10,Michael Brown,0.28,0


In [3]:
df_sorted['quartile'] = pd.qcut(df_sorted['Default Probability'],q=4,labels=['Q1','Q2','Q3','Q4'])
df_sorted

,Borrower Name,Default Probability,Default Truth,quartile
4,Aditi Sharma,0.75,1,Q4
1,Raj Patel,0.67,1,Q4
12,Anil Kumar,0.62,0,Q4
2,Meera Gupta,0.56,1,Q4
13,Sarah Lee,0.52,1,Q3
5,John Smith,0.49,1,Q3
7,Emily Chen,0.44,0,Q3
9,Vivek Singh,0.39,0,Q3
0,Priya Rao,0.32,0,Q2
10,Michael Brown,0.28,0,Q2


In [5]:
df_groupd = df_sorted.groupby('quartile').apply(lambda x: pd.Series({
    'Min Prob':x['Default Probability'].min(),
    'Max prob':x['Default Probability'].max(),
    'Event':x['Default Truth'].sum(),
    'Non-Event':x['Default Truth'].count() - x['Default Truth'].sum(),
}))
df_groupd.reset_index(inplace=True)
df_groupd

/var/folders/n6/28ff8fsj0c901jvkhvfb8ylw0000gn/T/ipykernel_1719/1219264085.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_groupd = df_sorted.groupby('quartile').apply(lambda x: pd.Series({
/var/folders/n6/28ff8fsj0c901jvkhvfb8ylw0000gn/T/ipykernel_1719/1219264085.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_groupd = df_sorted.groupby('quartile').apply(lambda x: pd.Series({


,quartile,Min Prob,Max prob,Event,Non-Event
0,Q1,0.03,0.15,0.00,4.00
1,Q2,0.18,0.32,0.00,4.00
2,Q3,0.39,0.52,2.00,2.00
3,Q4,0.56,0.75,3.00,1.00


In [6]:
df_groupd.sort_values('quartile',ascending=False,inplace=True)
df_groupd

,quartile,Min Prob,Max prob,Event,Non-Event
3,Q4,0.56,0.75,3.00,1.00
2,Q3,0.39,0.52,2.00,2.00
1,Q2,0.18,0.32,0.00,4.00
0,Q1,0.03,0.15,0.00,4.00


In [7]:
df_groupd['Event_rate'] = (df_groupd['Event']/(df_groupd['Event']+df_groupd['Non-Event']))*100
df_groupd['Non-Event_rate'] = (df_groupd['Non-Event']/(df_groupd['Event']+df_groupd['Non-Event']))*100
df_groupd

,quartile,Min Prob,Max prob,Event,Non-Event,Event_rate,Non-Event_rate
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00


In [8]:
df_groupd['cum_Event'] = df_groupd['Event'].cumsum()
df_groupd['cum_non_Event'] = df_groupd['Non-Event'].cumsum()
df_groupd

,quartile,Min Prob,Max prob,Event,Non-Event,Event_rate,Non-Event_rate,cum_Event,cum_non_Event
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00,3.00,1.00
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00,5.00,3.00
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00,5.00,7.00
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00,5.00,11.00


In [9]:
df_groupd['cum_Event_rate'] = (df_groupd['cum_Event']/df_groupd['Event'].sum())*100
df_groupd['cum_non_Event_rate'] = (df_groupd['cum_non_Event']/df_groupd['Non-Event'].sum())*100
df_groupd

,quartile,Min Prob,Max prob,Event,Non-Event,Event_rate,Non-Event_rate,cum_Event,cum_non_Event,cum_Event_rate,cum_non_Event_rate
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00,3.00,1.00,60.00,9.09
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00,5.00,3.00,100.00,27.27
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00,5.00,7.00,100.00,63.64
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00,5.00,11.00,100.00,100.00


In [11]:
df_groupd['KS'] = abs(df_groupd['cum_Event_rate']-df_groupd['cum_non_Event_rate'])
df_groupd

,quartile,Min Prob,Max prob,Event,Non-Event,Event_rate,Non-Event_rate,cum_Event,cum_non_Event,cum_Event_rate,cum_non_Event_rate,KS
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00,3.00,1.00,60.00,9.09,50.91
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00,5.00,3.00,100.00,27.27,72.73
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00,5.00,7.00,100.00,63.64,36.36
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00,5.00,11.00,100.00,100.00,0.00
